**Importing libraries and loading previously generated csv's form scraping and the train set deatails csv from kaggle**

In [1]:
import pandas as pd
import numpy as np
import os

In [78]:
data = pd.read_csv("CSV's/Train_set.csv")
aux = pd.read_csv("CSV's/artists_by_movement.csv")

**Filtering auxiliary dataset just for painters and performing some basic EDA**

In [79]:
aux.head(3)

,Unnamed: 0,movement,name,birth,death,country,art_type
0,0,Gothic Art,Benedetto Antelami,1150,1230,Italian,Sculptor
1,1,Gothic Art,Mastro Guglielmo,1158,1165,Italian,Sculptor
2,0,The Early Renaissance,Giotto di Bondone,1267,1337,Italian,Painter


In [80]:
# Function to check if artist type is painter in the complete artists list by movement
def painter_check(i):
    if 'Painter' in i:
        return 1
    else:
        return 0    

aux['is_painter'] = aux.art_type.apply(painter_check)
aux = aux[aux.is_painter == 1]
aux = aux.drop(columns = ['Unnamed: 0','is_painter', 'art_type'])
aux.columns = ['movement', 'artist', 'birth', 'death', 'country']
# filtering and reordering the dataframe
aux = aux[['artist','movement','birth', 'death', 'country']]

aux.head(3)

,artist,movement,birth,death,country
2,Giotto di Bondone,The Early Renaissance,1267,1337,Italian
3,Bernardo Daddi,The Early Renaissance,1280,1348,Italian
4,Taddeo Gaddi,The Early Renaissance,1300,1366,Italian


**Let's now check the main dataset**

In [166]:
data.isna().sum()

artist              0
date            26363
genre            1254
pixelsx             0
pixelsy             0
size_bytes          0
source            167
style             986
title             122
artist_group        0
in_train            0
new_filename        0
dtype: int64

**We have some blank styles, let's try to fill them joining the tables.**

In [226]:
merge = data.merge(aux, on='artist', how='left')

In [227]:
merge['style'] = merge.movement.where(merge['style'].isnull(), merge['style'])

In [228]:
merge.isna().sum()

artist              0
date            26476
genre            1259
pixelsx             0
pixelsy             0
size_bytes          0
source            167
style             932
title             122
artist_group        0
in_train            0
new_filename        0
movement        75715
birth           75715
death           75715
country         75715
dtype: int64

**Managed to fix someartists, sadly will have to discard the ones without genre/style because they are completly useless to our usecase, let's also filter just the columns that we need**

In [229]:
merge = merge[~merge['style'].isna()]

# calculating a helper column to fill the nan years for the painting with the midlife age of the artist
merge['avg_year'] = round((merge.death+merge.birth)/2)
merge['date'] = merge.avg_year.where(merge['date'].isnull(), merge['date'])

# Filtering the columns to use
merge = merge[['date', 'artist','style','new_filename','in_train']]

In [230]:
merge.isna().sum()

date            20632
artist              0
style               0
new_filename        0
in_train            0
dtype: int64

**We fixed more than 6000 dates and styles, so for now we will save it for future processing color palletes and HSL.**

In [1]:
# saving our work to a csv, fo safegranting
# merge.to_csv("CSV's/Treated_artist_by_movement_paintings.csv")

**Chosen categories**
***
Due to time restrains i'll train just 15 categories with a subsampling of 2000 paintings of each period. In a future iteration a better model can be calculated, but for now this subsample has to sufice. For the color palette analysis though i'll be using all the dataset, ignore the movement and just focus on the year.

In [232]:
movements_list = merge['style'].value_counts().head(15)
movements_list = pd.DataFrame(movements_list).reset_index()['index']

**Sub sampling the dataset for 2000 paintings for each category**

In [233]:
sub_sampled = pd.DataFrame(columns = merge.columns)

In [234]:
for element in movements_list:
    tmp = merge[merge['style'] == element]
    if len(tmp)<2000:
        sub_sampled = pd.concat([sub_sampled,tmp], axis=0)
    else:
        sub_sampled = pd.concat([sub_sampled,tmp.sample(2000)], axis=0)

**Sub_sample dataframe is complete, exporting to csv**

In [235]:
sub_sampled.to_csv("CSV's/sub_sample.csv")

**Creating the category folders**

In [236]:
for el in movements_list:
    os.mkdir('Model/Images/'+el)

In [238]:
movements_list.to_csv('Model_classes.csv')

**Moving files for their respective folders**

In [242]:
for i in sub_sampled.itertuples():
    destination = 'Model/Images/' + i[-3] +'/' + i[-2]
    if i[-1] == 1:
        shutil.copy2('E:/train/'+ i[-2], destination)
    else:
        shutil.copy2('E:/test/'+i [-2], destination)

**Checking for invalid files**

In [2]:
sub_sampled = pd.read_csv("CSV's/sub_sample.csv")

In [24]:
lst = []
from PIL import Image

# I try to open the images in the train dataset with pil, if he can't they are corrupted so i add them to
# a list of files to remove
for i,j in zip(sub_sampled['style'], sub_sampled.new_filename):
    path='Model/Images/'+i+'/'+j
    try:
        Image.open(path).load()
    except Exception as e:
        lst.append(path)

In [27]:
import os

lst_notdel = []

# Removing the corrupt files
for file in lst:
    try:
        os.remove(file) 
        print(file,'removed')
    except:
        lst_notdel.append(file)

Model/Images/Baroque/92899.jpg removed
Model/Images/Naïve Art (Primitivism)/18649.jpg removed
Model/Images/Cubism/41945.jpg removed
Model/Images/Cubism/101947.jpg removed
Model/Images/Cubism/100532.jpg removed
